# Monads

>Monads are monoids in the category of endofunctors 

(lol). Let's break this down

## Monoids

A monoid is a mathematical construct that holds a combination function between 2 values. Put simply, it's a unit of work where 2 values go in and one value comes out.

The empty method returns a base state where nothing is changed.

Combine is important as it is what makes the thing monadic. It needs to be associative. This means that the order of the arguments in the transformations does not matter, _only_ the order of the transformations.

For example, combine(a,combine(b,c)) is the same as combine(combine(a,b),c). This is because the evaluated result at the end will not change as the inputs are the same.

In [3]:
trait Monoid[T] {
    def empty: T
    def combine(a: T, b: T): T
}

defined trait Monoid

Let's look at an example

In [4]:
object IntMonoidAdd extends Monoid[Int] {
    override def empty = 0
    override def combine(a: Int, b: Int) = a + b
}

defined object IntMonoidAdd

In [5]:
val res = IntMonoidAdd.combine(2,3)

res: Int = 5

The monoid wraps the work we want to do (adding 2 numbers) The combine method derives the value from the inputs. If we were to use lots of monoid adds together, we would get the same results.

In [8]:
val res2 = IntMonoidAdd.combine(2, IntMonoidAdd.combine(3, IntMonoidAdd.combine(4,5)))
val res3 = IntMonoidAdd.combine(IntMonoidAdd.combine(2,5), IntMonoidAdd.combine(4,3))

println(res2 == res3)

true


res2: Int = 14
res3: Int = 14

This shows the associative nature of the monoid. Order of transformations matter, not inputs. (The above does the MonoidAdd 3 times in both scenarios with the same arguments, the args could be in different orders and the result would be the same)

### Functional Monoid

Monoids are simple, but they aren't the full story. Monoids can have many more ops than just combine and empty, those these are the defaults usually. Let's abstract this.

In [9]:
trait FunctionalMonoid[T] {
    def empty: Unit => T
    def combine: ((T, T)) => T
}

defined trait FunctionalMonoid

The functional monoid is identical to the above regarding function. The difference is that the detail of the operations is revealed more clearly (empty is a func that takes no args and returns a val)

Combine's API has changed though, it was a 2  arg func, but is now a 1 arg function. The FunctionalMonoid shows the Commutativeness of monads. Associative is order of transformations mattering. Commutativeness is the order of inputs _not_ mattering.

It doesn't matter what order the args are for the combine function in a monad. The result will always be the same if the transformations are the same.

### Very generic Monoid

In [10]:
trait GeneralMonoid[T, U, P] {
    def empty: U => T
    def combine: P => T
}

defined trait GeneralMonoid

- T is the type, as per usual
- U is the 0 or base state value of the monoid
- P is the product algebraic type that is consumed by combine


A key difference across the more abstract monoids is that they return functions, not values from their methods. This means their APIs can follow a lambda style approach rather than having to adhere to a specific contract. This is exemplified if we generify the monoid _even_ more

In [11]:
trait MostGeneralMonoid[T, ~>[_, _], U, P] {
    def empty: U ~> T
    def combine: P ~> T
}

defined trait MostGeneralMonoid

What this monoid does is interesting. We've extracted out the concept of a function into it's own type (this is a higher kinded type). The function type we've created can take up to 2 arguments, but it does not have to take any, or at the very least, we don't have to care about what the arguments are. This means we can use this higher kinded function for both the empty and combine opps.

The tricky thing is that the function is used infix, that is, it takes things either side of it as its arguments.

Using this most general monoid, we can remake our initial monoid from the beginning.

In [13]:
trait NewMonoid[T] extends MostGeneralMonoid[T, Function1, Unit, (T, T)]

defined trait NewMonoid

In [14]:
object IntMonoidMinus extends NewMonoid[Int] {
    override def empty = _ => 0
    override def combine = (a, b) => a - b
}

defined object IntMonoidMinus

In [15]:
val res4 = IntMonoidMinus.combine(7,2)

res4: Int = 5

### Higher kinded general monoid

In [16]:
trait MostGeneralMonoidK2[T[_], ~>[_[_], _[_]], U[_], P[_]] {
    def empty: U ~> T
    def combine: P ~> T
}

defined trait MostGeneralMonoidK2

The above is a higher kinded version of the most general monoid. The most general monoid was in the category of type T. This means that it works on things of type T only.

The above is in the category of `T[_]`

This means anything that is wrapped with T can be worked on by this monoid

## Functors

>A function that defines a transformation between categories

Let's break this down too

In [17]:
trait Functor[F[_]] {
    def map[A, B](fa: F[A])(fn: A => B): F[B]
}

defined trait Functor

- A is a type
- B is a type

F is a generic value holder

The trait describes a transformation from one class to another (in scala a category could be thought of as synonymous with a class)

`fa` maps the function holder to A. `fn` describes the transformation between A and B. The return is F wrapped with B as that is the new value after the transformation has occurred.

The definition is self contained. This is what makes it 'pure'

An endofunctor is a functor that ingests a category and returns that same category. For example a functor that worked on Option and returns another Option

### Functor transformations

A functor transformation is the opposite of an endofunctor and is defined as follows

In [18]:
trait FunctorTransformation[-F[_], +G[_]] {
    def apply[A](fa: F[A]): G[A]
}

defined trait FunctorTransformation

Note how in this example the value doesn't change but the wrapping type does. Functors can also work on categories too. This is important as we often need to change how something works and this is how we do that.

The variance annotations are significant here as the F type must be specific, otherwise the transformation could fail unexpectedly (only the type specified for the transformation is allowed). The G type can be covariant as anything that is in the same category is fine (Anything that matches the box we want is fine).

F and G can be data wrapper types like: List, Option: Either.

These are implemented Functors

### ID functor